### 1- Adding fasttext probs and sentiments

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("../data/repos/3/allrepos_processed.csv")
senti = pd.read_csv("../data/repos/3/allrepos_sentiments.csv")
ftprobs = pd.read_csv("../data/repos/3/allrepos_ftprobs.csv")

In [3]:
df["ft_bug"] = ftprobs["__label__bug"] 
df["ft_feature"] = ftprobs["__label__feature"] 
df["ft_other"] = ftprobs["__label__other"] 

In [4]:
df["title_sentistrenght_p"] = senti["title_sentistrenght"].apply(lambda x: x.split(',')[0])
df["title_sentistrenght_n"] = senti["title_sentistrenght"].apply(lambda x: x.split(',')[1])

df["body_sentistrenght_p"] = senti["body_sentistrenght"].apply(lambda x: x.split(',')[0])
df["body_sentistrenght_n"] = senti["body_sentistrenght"].apply(lambda x: x.split(',')[1])

df["title_polarity"] = senti["title_textblob"].apply(lambda x: x.split(',')[0])
df["title_subjectivity"] = senti["title_textblob"].apply(lambda x: x.split(',')[1])

df["body_polarity"] = senti["body_textblob"].apply(lambda x: x.split(',')[0])
df["body_subjectivity"] = senti["body_textblob"].apply(lambda x: x.split(',')[1])

In [5]:
df["positive_title_sentistrenght_n"] = df["title_sentistrenght_n"].astype(int).abs()
df["positive_body_sentistrenght_n"] = df["body_sentistrenght_n"].astype(int).abs()
df["positive_title_polarity"] = df["title_polarity"].astype(float) + 1
df["positive_body_polarity"] = df["body_polarity"].astype(float) + 1

### 2- Applynig some other preprocessings and adding some columns

In [6]:
df = df[~df.closer_login.isna()]

In [7]:
def compute_time_interval(t1, t2):
    d1 = datetime.strptime(t1, "%Y-%m-%dT%H:%M:%SZ")
    d2 = datetime.strptime(t2, "%Y-%m-%dT%H:%M:%SZ")
    delta = d2-d1
    return delta.days

In [8]:
base_time = df.created_at.max()

In [9]:
df["author_account_age"] = df["author_created_at"].apply(lambda x: round(compute_time_interval(x, base_time)/365))    
df["closer_account_age"] = df["closer_created_at"].apply(lambda x: round(compute_time_interval(x, base_time)/365))

In [10]:
df["has_assignee"] = ~df["assignee"].isna()
df["num_of_assignees"] = df["assignees"].apply(lambda x: 0 if pd.isna(x) else len(x.split("|")))
df['has_milestone'] = df['milestone'].apply(lambda x: 0 if pd.isna(x) else 1) 

In [11]:
df["author_github_cntrb"] = df["author_github_cntrb"].apply(lambda x: int(str(x).replace(',','')))
df["closer_github_cntrb"] = df["closer_github_cntrb"].apply(lambda x: int(str(x).replace(',','')))

In [12]:
df['numeric_association'] = df['author_association'].apply(lambda x: 0 if x == "NONE"  else 1 if x == "CONTRIBUTOR" else 2 if x == "MEMBER" else 3 if x == "OWNER" else 4)

In [13]:
# print(df[(df.repository == "spring-framework") & (df.author_login == "spring-issuemaster")].shape)
# print(df[(df.repository == "spring-framework") & (df.author_login != "spring-issuemaster")].shape)

In [14]:
# print(df[(df.repository == "spring-framework") & (df.closer_login == "spring-issuemaster")].shape)
# print(df[(df.repository == "spring-framework") & (df.closer_login != "spring-issuemaster")].shape)

In [15]:
# print(df[(df.repository == "spring-framework") & (df.author_login == "spring-issuemaster") & (df.closer_login == "spring-issuemaster")].shape)

In [16]:
# df["author_creation_day_before_issue"] = df.apply(lambda x: np.nan if x.author_login == "spring-issuemaster" else compute_time_interval(x.author_created_at, x.created_at) ,axis=1)
# df["tyro_author"] = df["author_creation_day_before_issue"].apply(lambda x: 1 if x<5 else 0)

In [ ]:
'''
spring-framework has a spring-issuemaster author which owns 17005 issues form total 19719 issues!!
this user also closed 16900 issues of spring-framework and 230 issues from spring-boot
spring-issuemaster doesnt have any repo and star, thus considering author info for spring-framework repo is meaningless
'''

In [17]:
# df.columns[df.isna().any()].tolist()

In [18]:
df[df.reaction_time.isna()].shape

(22019, 112)

In [19]:
dataframes = {}
dataframes["elasticsearch"] =  df[df.repository == "elasticsearch"]
dataframes["spring-boot"] =  df[(df.repository == "spring-boot") & (df.closer_login != "spring-issuemaster")]
dataframes["spring-framework"] =  df[(df.repository == "spring-framework") & (df.author_login == "spring-issuemaster") & (df.closer_login == "spring-issuemaster")]
dataframes["cross-project"] = df

In [20]:
repos = list(dataframes.keys())
for repo in repos:
    dataframes[repo].to_csv(f"../data/repos/final/{repo}.csv", index=False)

In [21]:
# for repo in ["elasticsearch", "spring-boot", "spring-framework", "cross-project"]:
#     df = pd.read_csv(f"../data/repos/final/{repo}.csv")
#     print(repo, df.shape)

In [22]:
# 16876+15025+44099

### 3- Normalization

In [33]:
nontext_columns = [    
    'is_pull_request',
    'title_len',
    'body_len',
    'num_comments',
    'num_events',
    'author_followers',
    'closer_followers',
    'author_following',
    'closer_following',
    'author_public_repos',
    'closer_public_repos',
    'author_public_gists',
    'closer_public_gists',
    'author_core_team',
    'author_has_association',
    'author_issue_counts',
    'commits_count',
    'has_commit',
    'cm_developers_number',
    'cm_developers_ratio',
    'cm_developers_unique',
    'cm_authors_unique',
    'cm_developers_ratio_unique',
    'cm_mean_len',
    'time_to_discuss',
    'author_github_cntrb',
    'closer_github_cntrb',
    'author_repo_cntrb',
    'closer_repo_cntrb',
    'title_words_num',
    'body_words_num',   
    'title_alpha_len',
    'title_alphabet_ratio',
    'body_alpha_len',
    'body_alphabet_ratio',
    'body_processed_len',
    'title_processed_len',
    'title_processed_words_num',
    'body_processed_words_num',
    'num_of_sharps',
    'num_of_at',
    'num_of_qmark',
    'num_of_codesnippets',
    'num_of_functions',
    'num_of_issues',
    'num_of_paths',
    'num_of_dates',
    'num_of_times',
    'num_of_urls',
    'num_of_emails',
    'num_of_obligations',
    'has_email',
    'has_code',
    'ft_bug',
    'ft_feature',
    'ft_other',    
    'title_sentistrenght_p',
    'body_sentistrenght_p',
    'title_subjectivity',
    'body_subjectivity',
    'positive_body_sentistrenght_n',
    'positive_title_sentistrenght_n',
    'positive_title_polarity',
    'positive_body_polarity',
    'author_account_age',
    'closer_account_age',
    'has_assignee',
    'num_of_assignees',
    'has_milestone',
    'numeric_association'
]

In [34]:
for repo in repos:
    print(repo)
    df = dataframes[repo]
    train, test = train_test_split(df, test_size=0.2, random_state = 42, shuffle=True)
    train["test_tag"] = 0
    test["test_tag"] = 1
    min_max_scaler = MinMaxScaler()
    train[nontext_columns]  = min_max_scaler.fit_transform(train[nontext_columns])
    test[nontext_columns]  = min_max_scaler.transform(test[nontext_columns])
    df = pd.concat([train, test], ignore_index=True)
    df.to_csv(f"../data/repos/final/{repo}_norm.csv", index=False)

elasticsearch
spring-boot
spring-framework
cross-project
